---
format:
  html:
    code-line-numbers: true
    code-overflow: wrap
    code-block-bg: true
    code-block-border-left: true
    highlight-style: Arrow
---

# Benders Decomposition

In this chapter, we will explain the theories behind Benders decomposition and demonstrate its usage on a trial linear programming problem.
Keep in mind that Benders decomposition is not limited to solving linear programming problems.
In fact, it is one of the most powerful techniques to solve some large-scale mixed-integer linear programming problems.

In the following sections, we will go through the critical steps during the decomposition process when applying the algorithm on optimization problems represented in standard forms.
This is important as it helps build up the intuition of when we should consider applying Benders decomposition to a problem at hand.
Often times, recognizing the applicability of Benders decomposition is the most important and challenging step when solving an optimization problem.
Once we know that the problem structure is suitable to solve via Benders decomposition, it is straightforward to follow the decomposition steps and put it into work.

Generally speaking, Benders decomposition is a good solution candidate when the resulting problem is much easier to solve if some of the variables in the original problem are fixed.
We will illustrate this point using an example in the following sections.
In the optimization world, the first candidate that should come to mind when we say a problem is easy to solve is a linear programming formulation, which is indeed the case in Benders decomposition applications.

## The Decomposition Logic

To explain the reasoning of Benders decomposition, let us look at the standard form of linear programming problems that involve two vector variables, $\mathbf{x}$ and $\mathbf{y}$. Let $p$ and $q$ indicate the dimensions of $\mathbf{x}$ and $\mathbf{y}$, respectively.
Below is the original problem $\mathbf{P}$ we intend to solve.

\begin{align}
&(\mathbf{P}) &\quad \text{min.} &\quad \mathbf{c}^T \mathbf{x} + \mathbf{f}^T \mathbf{y} \\
& &\quad \text{s.t.} &\quad \mathbf{A} \mathbf{x} + \mathbf{B} \mathbf{y} = \mathbf{b} \\
& &\quad &\quad \mathbf{x} \geq 0, \mathbf{y} \geq 0
\end{align}

In this formulation, $\mathbf{c}$ and $\mathbf{f}$ in the objective function represent the cost coefficients associated with decision variables $\mathbf{x}$ and $\mathbf{y}$, respectively.
Both of them are column vectors of corresponding dimensions.
In the constraints, matrix $\mathbf{A}$ is of dimension $m \times p$, and matrix $\mathbf{B}$ is of dimension $m \times q$.
$\mathbf{b}$ is a column vector of dimension $m$.

Suppose the variable $\mathbf{y}$ is a complicating variable in the sense that the resulting problem is substantially easier to solve if the value of $\mathbf{y}$ is fixed.
In this case, we could rewrite problem $\mathbf{P}$ as the following form:

\begin{align}
\text{min.} &\quad \mathbf{f}^T \mathbf{y} + g(\mathbf{y}) \\
\text{s.t.} &\quad \mathbf{y} \geq 0
\end{align}

where $g(\mathbf{y})$ is a function of $\mathbf{y}$ and is defined as the subproblem $\mathbf{SP}$ of the form below:

\begin{align}
&(\mathbf{SP}) &\quad \text{min.} &\quad \mathbf{c}^T \mathbf{x} \\
& &\quad \text{s.t.} &\quad \mathbf{A} \mathbf{x}  = \mathbf{b} - \mathbf{B} \mathbf{y} \label{bd-cons1} \\
& &\quad &\quad \mathbf{x} \geq 0
\end{align}

Note that the $\mathbf{y}$ in constraint \eqref{bd-cons1} takes on some known values when the problem is solved and the only decision variable in the above formulation is $\mathbf{x}$.
The dual problem of $\mathbf{SP}$, $\mathbf{DSP}$, is given below.


\begin{align}
&(\mathbf{DSP}) &\quad \text{max.} &\quad (\mathbf{b} - \mathbf{B} \mathbf{y})^{T} \mathbf{u} \\
& &\quad \text{s.t.} &\quad \mathbf{A}^T \mathbf{u} \leq \mathbf{c} \label{bd-cons2} \\
& &\quad &\quad \mathbf{u}\  \text{unrestricted}
\end{align}

A key characteristic of the above $\mathbf{DSP}$ is that its solution space does not depend on the value of $\mathbf{y}$, which only affects the objective function.
According to the Minkowski’s representation theorem, any $\bar{\mathbf{u}}$ satisfying the constraints \eqref{bd-cons2} can be expressed as

\begin{align}
\bar{\mathbf{u}} = \sum_{j \in \mathbf{J}} \lambda_j \mathbf{u}_{j}^{point} + \sum_{k \in \mathbf{K}} \mu_k \mathbf{u}_k^{ray}
\end{align}

where $\mathbf{u}_j^{point}$ and $\mathbf{u}_k^{ray}$ represent an extreme point and extreme ray, respectively.
In addition, $\lambda_j \geq 0$ for all $j \in \mathbf{J}$ and $\sum_{j \in \mathbf{J}}\lambda_j = 1$, and $\mu_k \geq 0$ for all $k \in \mathbf{K}$.
It follows that the $\mathbf{DSP}$ is equivalent to 

\begin{align}
\text{max.} &\quad (\mathbf{b} - \mathbf{B} \mathbf{y})^{T} (\sum_{j \in \mathbf{J}} \lambda_j \mathbf{u}_{j}^{point} + \sum_{k \in \mathbf{K}} \mu_k \mathbf{u}_k^{ray}) \\
\text{s.t.} &\quad \sum_{j \in \mathbf{J}}\lambda_j = 1 \\
&\quad \lambda_j \geq 0, \ \forall j \in \mathbf{J} \\
&\quad \mu_k \geq 0, \ \forall k \in \mathbf{K}
\end{align}

We can therefore conclude that

- The $\mathbf{DSP}$ becomes unbounded if any $\mathbf{u}_k^{ray}$ exists such that $(\mathbf{b} - \mathbf{B} \mathbf{y})^{T} \mathbf{u}_k^{ray} > 0$.
Note that an unbounded $\mathbf{DSP}$ implies an infeasible $\mathbf{SP}$ and to prevent this from happening, we have to ensure that $(\mathbf{b} - \mathbf{B} \mathbf{y})^{T} \mathbf{u}_k^{ray} \leq 0$ for all $k \in \mathbf{K}$.
- If an optimal solution to $\mathbf{DSP}$ exists, it must occur at one of the extreme points. Let $g$ denote the optimal objective value, it follows that $(\mathbf{b} - \mathbf{B} \mathbf{y})^{T} \mathbf{u}_j^{point} \leq g$ for all $j \in \mathbf{J}$.

Based on this idea, the $\mathbf{DSP}$ can be reformulated as follows:

\begin{align}
\text{min.} &\quad g \\
\text{s.t.} &\quad (\mathbf{b} - \mathbf{B} \mathbf{y})^{T} \mathbf{u}_k^{ray} \leq 0, \ \forall j \in \mathbf{J} \label{bd-feas} \\
&\quad (\mathbf{b} - \mathbf{B} \mathbf{y})^{T} \mathbf{u}_j^{point} \leq g, \ \forall k \in \mathbf{K} \label{bd-opt} \\
&\quad j \in \mathbf{J}, k \in \mathbf{K}
\end{align}

Constraints \eqref{bd-feas} are called **Benders feasibility cuts**, while constraints \eqref{bd-opt} are called **Benders optimality cuts**.
Now we are ready to define the Benders Master Problem ($\mathbf{BMP}$) as follows:

\begin{align}
&(\mathbf{BMP}) &\quad \text{min.} &\quad \mathbf{f}^T \mathbf{y} + g \\
& &\quad \text{s.t.} &\quad (\mathbf{b} - \mathbf{B} \mathbf{y})^{T} \mathbf{u}_k^{ray} \leq 0, \ \forall j \in \mathbf{J} \\
& &\quad &\quad (\mathbf{b} - \mathbf{B} \mathbf{y})^{T} \mathbf{u}_j^{point} \leq g, \ \forall k \in \mathbf{K} \\
& &\quad &\quad j \in \mathbf{J}, k \in \mathbf{K}, \mathbf{y} \geq 0
\end{align}

Typically $J$ and $K$ are too large to enumerate upfront and we have to work with subsets of them, denoted by $J_s$ and $K_s$, respectively. Hence we have the following Restricted Benders Master Problem ($\mathbf{RBMP}$):

\begin{align}
&(\mathbf{RBMP}) &\quad \text{min.} &\quad \mathbf{f}^T \mathbf{y} + g \\
& &\quad \text{s.t.} &\quad (\mathbf{b} - \mathbf{B} \mathbf{y})^{T} \mathbf{u}_k^{ray} \leq 0, \ \forall j \in \mathbf{J}_s \\
& &\quad &\quad (\mathbf{b} - \mathbf{B} \mathbf{y})^{T} \mathbf{u}_j^{point} \leq g, \ \forall k \in \mathbf{K}_s \\
& &\quad &\quad j \in \mathbf{J}, k \in \mathbf{K}, \mathbf{y} \geq 0
\end{align}

```{mermaid}
%%| label: fig-benders-flowchart
%%| fig-cap: Benders decomposition workflow

flowchart LR
   A[Start] --> B{Is}
    B -->|Yes| C[OK]
    C --> D[Rethink]
    D --> B
    B ---->|No| E[End]
```

## A linear programming example

In this section, we will first present a small linear programming problem and solve it directly using the Gurobi API in Python - *gurobipy*.
Then we will demonstrate the Benders decomposition approach on this artificial problem.
Lastly, we will provide an implementation to solve this problem in gurobipy.

### The original problem and its optimal solution

The linear program we examine here is devoid of any practical meaning and is solely used to demonstrate the solution process of Benders decomposition.
The problem is stated below, in which $\mathbf{x} = (x_1, x_2, x_3)$ and $\mathbf{y} = (y_1, y_2)$ are the decision variables.
We assume that $\mathbf{y}$ is the complicating variable.

\begin{align*}
    \text{min.} &\quad 8x_1 + 12x_2 +10x_3 + 15y_1 + 18y_2 \\
    \text{s.t.} &\quad 2x_1 + 3x_2 + 2x_3 + 4y_1 + 5y_2 = 300 \\
    &\quad 4x_1 + 2x_2 + 3x_3 + 2y_1 + 3y_2 = 220 \\
    &\quad x_i \geq 0, \ \forall i = 1, \cdots, 3 \\
    &\quad y_i \geq 0, \ \forall j = 1, 2
\end{align*}

In this example, $\mathbf{c}^T = (8, 12, 10)$, $\mathbf{f}^T = (15, 18)$ and $\mathbf{b}^T = (300, 220)$.
In addition,

\begin{equation*}
\mathbf{A} = 
\begin{bmatrix}
    2 & 3 & 2 \\
    4 & 2 & 3 \\
\end{bmatrix}
\qquad
\mathbf{B} = 
\begin{bmatrix}
    4 & 5 \\
    2 & 3 \\
\end{bmatrix}
\end{equation*}

We first use Gurobi to identify its optimal solution.

In [ ]:
#| echo: true
#| output: false

import gurobipy as gp
from gurobipy import GRB

# Create a new model
env = gp.Env(empty=True)
env.setParam('OutputFlag', 0)
env.start()
model = gp.Model(env=env, name="original_problem")

# Decision variables
x1 = model.addVar(vtype=GRB.CONTINUOUS, name='x1')
x2 = model.addVar(vtype=GRB.CONTINUOUS, name='x2')
x3 = model.addVar(vtype=GRB.CONTINUOUS, name='x3')
y1 = model.addVar(vtype=GRB.CONTINUOUS, name='y1')
y2 = model.addVar(vtype=GRB.CONTINUOUS, name='y2')

# Objective function
model.setObjective(8*x1 + 12*x2 + 10*x3 + 15*y1 + 18*y2, 
                   GRB.MINIMIZE)

# Constraints
model.addConstr(2*x1 + 3*x2 + 2*x3 + 4*y1 + 5*y2 == 300)
model.addConstr(4*x1 + 2*x2 + 3*x3 + 2*y1 + 3*y2 == 220)

# Optimize the model
model.optimize()

# Print the results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found!")
    print(f'x1 = {x1.X:.2f}')
    print(f'x2 = {x2.X:.2f}')
    print(f'x3 = {x3.X:.2f}')
    print(f'y1 = {y1.X:.2f}')
    print(f'y2 = {y2.X:.2f}')
    print(f"Total cost: {model.objVal:.2f}")
else:
    print("No solution found.")

# Close the Gurobi environment
model.dispose()
env.dispose()


The optimal solution and objective value are as follows.

```{{python}}
Optimal solution found!
x1 = 14.29
x2 = 0.00
x3 = 0.00
y1 = 0.00
y2 = 54.29
Total cost: 1091.43
```

### Benders decomposition

We first state the subproblem as follows:

\begin{align*}
    &(\mathbf{SP}) &\quad \text{min.} &\quad 8x_1 + 12x_2 +10x_3 \\
    &&\quad \text{s.t.} &\quad 2x_1 + 3x_2 + 2x_3 = 300 - 4y_1 - 5y_2 \\
    &&&\quad 4x_1 + 2x_2 + 3x_3 = 220 - 2y_1 - 3y_2 \\
    &&&\quad x_i \geq 0, \ \forall i = 1, \cdots, 3
\end{align*}

We define two dual variables $u_1$ and $u_2$ to associate with the two constraints in the subproblem.
The dual subproblem could then be stated as follows:

\begin{align*}
    &(\mathbf{DSP}) &\quad \text{max.} &\quad (300 - 4y_1 - 5y_2) u_1 + (220 - 2y_1 - 3y_2) u_2 \\
    &&\quad \text{s.t.} &\quad 2u_1 + 4u_2 \leq 8\\
    &&&\quad 3u_1 + 2u_2 \leq 12 \\
    &&&\quad 2u_1 + 3u_2 \leq 10 \\
    &&&\quad u_1, u_2\  \text{unrestricted}
\end{align*}

The *RBMP* can be stated as:

\begin{align*}
    &(\mathbf{RBMP}) &\quad \text{min.} &\quad 15 y_1 + 18 y_2 + g \\
    &&\quad \text{s.t.} &\quad  y_1, y_2 \geq 0 \\
    &&&\quad g \leq 0
\end{align*}

### Solving the problem step by step

In this section, we will solve the linear program step by step using Gurobi.
To this end, we first import the necessary libraries and create an environment `env`.

In [ ]:
#｜ output: false
import numpy as np
import gurobipy as gp
from gurobipy import GRB

env = gp.Env('benders')
env.setParam('OutputFlag', 0)

Next, we initialize several algorithm parameters, specifically, we use `lb` and `ub` to represent the lower and upper bounds of the solution.
The `eps` is defined as a small number to decide whether the searching process should stop.

The remaining codes aim to create the restricted master Benders problem indicated by `rbmp`.
Note that it only has the $y$ and $g$ variables and the objective function, there is no constraint added to the model yet.

In [ ]:
# parameters
lb = -GRB.INFINITY
ub = GRB.INFINITY
eps = 1.0e-5

# create restricted Benders master problem
rbmp = gp.Model(env=env, name='RBMP')

# create decision variables
y1 = rbmp.addVar(vtype=GRB.CONTINUOUS, lb=0, name='y1')
y2 = rbmp.addVar(vtype=GRB.CONTINUOUS, lb=0, name='y2')
g = rbmp.addVar(vtype=GRB.CONTINUOUS, lb=0, name='g')

# create objective
rbmp.setObjective(15*y1 + 18*y2 + g, GRB.MINIMIZE)

We then define the model in Gurobi to solve the dual subproblem, represented by `dsp`.
It consists of two decision variables `u1` and `u2`.
The constraints are created in lines 12 - 14.

In [ ]:
# create dual subproblem
dsp = gp.Model(env=env, name='DSP')

# create decision variables
u1 = dsp.addVar(vtype=GRB.CONTINUOUS, name='u1')
u2 = dsp.addVar(vtype=GRB.CONTINUOUS, name='u2')

# create objective function
dsp.setObjective(300*u1 + 220*u2)

# create constraints
dsp.addConstr(2*u1 + 4*u2 <= 8, name='c1')
dsp.addConstr(3*u1 + 2*u2 <= 12, name='c2')
dsp.addConstr(2*u1 + 3*u2 <= 10, name='c3')

dsp.update()

In the very first iteration,  we solve the **RBMP**, as shown in the following code snippet.

In [ ]:
rbmp.optimize()

if rbmp.status == GRB.OPTIMAL:
    print(f'optimal solution found!')
    
    y1_opt = y1.X
    y2_opt = y2.X
    g_opt = g.X
    lb = np.max([lb, rbmp.objVal])
    
    print(f'optimal obj: {rbmp.objVal:.2f}')
    print(f'y1 = {y1_opt:.2f}')
    print(f'y2 = {y2_opt:.2f}')
    print(f'g = {g_opt:.2f}')
    print(f'lb={lb}, ub={ub}')
elif rbmp.status == GRB.INFEASIBLE:
    print(f'original problem is infeasible!')

Now we have obtained an optimal solution $(\bar{y_1}, \bar{y_2}, \bar{g}) = (0, 0, 0)$, which also provides a new lower bound to our problem.
We now feed the values of $\bar{y_1}$ and $\bar{y_2}$ into the Benders subproblem **(SP)**:

In [ ]:

dsp.setObjective((300-4*y1_opt-5*y2_opt)*u1 + 
                 (220-2*y1_opt-3*y2_opt)*u2, 
                 GRB.MAXIMIZE)
dsp.update()
dsp.optimize()

if dsp.status == GRB.OPTIMAL:
    u1_opt = u1.X
    u2_opt = u2.X
    
    print(f'optimal obj = {dsp.objVal:.2f}')
    print(f'u1 = {u1_opt:.2f}')
    print(f'u2 = {u2_opt:.2f}')
    ub = np.min([ub, 15*y1_opt + 18*y2_opt + dsp.objVal])
    print(f'lb={lb}, ub={ub}')
elif dsp.Status == GRB.UNBOUNDED:
    # add feasibility cut
    pass
else:
    pass

We see that the dual subproblem has an optimal solution.
Note that in line 15, the upper bound of the problem is updated.

Since the optimal objective value of the subproblem turns out to be 1200 and is greater than $\bar{g} = 0$, which implies that an optimality cut is needed to make sure that the variable $g$ in the restricted Benders master problem reflects this newly obtained information from the subproblem.

In [ ]:
rbmp.addConstr((300-4*y2-5*y2)*u1_opt 
               + (220-2*y1-3*y2)*u2_opt <= g, 
               name='c3')
rbmp.update()
rbmp.optimize()

if rbmp.status == GRB.OPTIMAL:
    print(f'optimal solution found!')
    
    y1_opt = y1.X
    y2_opt = y2.X
    g_opt = g.X
    lb = np.max([lb, rbmp.objVal])
    
    print(f'optimal obj: {rbmp.objVal:.2f}')
    print(f'y1 = {y1_opt:.2f}')
    print(f'y2 = {y2_opt:.2f}')
    print(f'g = {g_opt:.2f}')
    print(f'lb={lb}, ub={ub}')
elif rbmp.status == GRB.INFEASIBLE:
    print(f'original problem is infeasible!')

Now we solve the subproblem again with the newly obtained solution $(\bar{y_1}, \bar{y_2}, \bar{g}) = (0, 33.33, 0)$.

In [ ]:
dsp.setObjective((300 - 4*y1_opt - 5*y2_opt) * u1 
                 + (220 - 2*y1_opt - 3*y2_opt) * u2, 
                 GRB.MAXIMIZE)
dsp.update()
dsp.optimize()

if dsp.status == GRB.OPTIMAL:
    u1_opt = u1.X
    u2_opt = u2.X
    
    print(f'optimal obj = {dsp.objVal:.2f}')
    print(f'u1 = {u1_opt:.2f}')
    print(f'u2 = {u2_opt:.2f}')
    ub = np.min([ub, 15*y1_opt + 18*y2_opt + dsp.objVal])
    print(f'lb={lb}, ub={ub}')
elif dsp.status == GRB.UNBOUNDED:
    print(f'dual subproblem is unbounded!')

Since the optimal objective value of the subproblem, 533.33, is still bigger than $\bar{g} = 0$, an optimality cut is needed.
In the below code snippet, we add the new cut and solve the restricted Benders master problem again.

In [ ]:
rbmp.addConstr((300 - 4*y1 - 5*y2) * u1_opt + (220 - 2*y1 - 3*y2) * u2_opt <= g, name='c3')
rbmp.update()
rbmp.optimize()

if rbmp.status == GRB.OPTIMAL:
    print(f'optimal solution found!')
    
    y1_opt = y1.X
    y2_opt = y2.X
    g_opt = g.X
    lb = np.max([lb, rbmp.objVal])
    
    print(f'optimal obj: {rbmp.objVal:.2f}')
    print(f'y1 = {y1_opt:.2f}')
    print(f'y2 = {y2_opt:.2f}')
    print(f'g = {g_opt:.2f}')
    print(f'lb={lb}, ub={ub}')
elif rbmp.status == GRB.INFEASIBLE:
    print(f'original problem is infeasible!')

Note that a new lower bound is obtained after solving the master problem.
Since there is still a large gap between the lower bound and upper bound, we continue solving the subproblem.

In [ ]:
dsp.setObjective((300 - 4*y1_opt - 5*y2_opt) * u1 + (220 - 2*y1_opt - 3*y2_opt) * u2, GRB.MAXIMIZE)
dsp.update()
dsp.optimize()

if dsp.status == GRB.OPTIMAL:
    u1_opt = u1.X
    u2_opt = u2.X
    
    print(f'optimal obj = {dsp.objVal:.2f}')
    print(f'u1 = {u1_opt:.2f}')
    print(f'u2 = {u2_opt:.2f}')
    ub = np.min([ub, 15*y1_opt + 18*y2_opt + dsp.objVal])
    print(f'lb={lb}, ub={ub}')
elif dsp.status == GRB.UNBOUNDED:
    print(f'dual subproblem is unbounded!')

Now the upper bound is reduced to 1133.33, but the subproblem optimal solution is still bigger than the value of $\bar{g} = 0$.

In [ ]:
rbmp.addConstr((300 - 4*y1 - 5*y2) * u1_opt + (220 - 2*y1 - 3*y2) * u2_opt <= g, name='c3')
rbmp.update()
rbmp.optimize()

if rbmp.status == GRB.OPTIMAL:
    print(f'optimal solution found!')
    
    y1_opt = y1.X
    y2_opt = y2.X
    g_opt = g.X
    lb = np.max([lb, rbmp.objVal])
    
    print(f'optimal obj: {rbmp.objVal:.2f}')
    print(f'y1 = {y1_opt:.2f}')
    print(f'y2 = {y2_opt:.2f}')
    print(f'g = {g_opt:.2f}')
    print(f'lb={lb}, ub={ub}')
elif rbmp.status == GRB.INFEASIBLE:
    print(f'original problem is infeasible!')

In [ ]:
dsp.setObjective((300 - 4*y1_opt - 5*y2_opt) * u1 + (220 - 2*y1_opt - 3*y2_opt) * u2, GRB.MAXIMIZE)
dsp.update()
dsp.optimize()

if dsp.status == GRB.OPTIMAL:
    u1_opt = u1.X
    u2_opt = u2.X
    
    print(f'optimal obj = {dsp.objVal:.2f}')
    print(f'u1 = {u1_opt:.2f}')
    print(f'u2 = {u2_opt:.2f}')
    ub = np.min([ub, 15*y1_opt + 18*y2_opt + dsp.objVal])
    print(f'lb={lb}, ub={ub}')
elif dsp.status == GRB.UNBOUNDED:
    print(f'dual subproblem is unbounded!')

Now the gap between the lower bound and upper bound is reduced to 0, the problem completes.

### Putting it together

Certainly we don't want to manually control the interaction between the master problem and subproblem to find the optimal solution.
Therefore, in this section, we will put every together to come up with a control flow to help us identify the optimal solution automatically.

In [2]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
from enum import Enum

class OptStatus(Enum):
    OPTIMAL = 0
    UNBOUNDED = 1
    INFEASIBLE = 2
    ERROR = 3

In [55]:
class MasterSolver:
    
    def __init__(self, env):
        self._model = gp.Model(env=env, name='RBMP')
        
        # create decision variables
        self._y1 = self._model.addVar(vtype=GRB.CONTINUOUS, lb=0, name='y1')
        self._y2 = self._model.addVar(vtype=GRB.CONTINUOUS, lb=0, name='y2')
        self._g = self._model.addVar(vtype=GRB.CONTINUOUS, lb=0, name='g')

        # create objective
        self._model.setObjective(15*self._y1 + 18*self._y2 + self._g, GRB.MINIMIZE)
        
        self._opt_obj = None
        self._opt_y1 = None
        self._opt_y2 = None
        self._opt_g = None
        
    def solve(self) -> OptStatus:
        print('-' * 50)
        print(f'Start solving master problem.')
        self._model.optimize()
        
        opt_status = None
        if self._model.status == GRB.OPTIMAL:
            opt_status = OptStatus.OPTIMAL
            self._opt_obj = self._model.objVal
            self._opt_y1 = self._y1.X
            self._opt_y2 = self._y2.X
            self._opt_g = self._g.X
            print(f'\tmaster problem is optimal.')
            print(f'\topt_obj={self._opt_obj:.2f}')
            print(f'\topt_y1={self._opt_y1:.2f}, opt_y2={self._opt_y2:.2f}, opt_g={self._opt_g:.2f}')
        elif self._model.status == GRB.INFEASIBLE:
            print(f'\tmaster problem is infeasible.')
            opt_status = OptStatus.INFEASIBLE
        else:
            print(f'\tmaster problem encountered error.')
            opt_status = OptStatus.ERROR
        
        print(f'Finish solving master problem.') 
        print('-' * 50)
        return opt_status
    
    def add_feasibility_cut(self, opt_u1, opt_u2) -> None:
        self._model.addConstr((300 - 4*self._y1 - 5*self._y2) * opt_u1 + 
                              (220 - 2*self._y1 - 3*self._y2) * opt_u2 <= 0)
        print(f'Benders feasibility cut added!')
    
    def add_optimality_cut(self, opt_u1, opt_u2) -> None:
        self._model.addConstr((300 - 4*self._y1 - 5*self._y2) * opt_u1 + 
                              (220 - 2*self._y1 - 3*self._y2) * opt_u2 <= self._g)
        print(f'Benders optimality cut added!')
    
    def clean_up(self):
        self._model.dispose()
        
    @property
    def opt_obj(self):
        return self._opt_obj
    
    @property
    def opt_y1(self):
        return self._opt_y1
    
    @property
    def opt_y2(self):
        return self._opt_y2
    
    @property
    def opt_g(self):
        return self._g

In [56]:
class DualSubprobSolver:
    
    def __init__(self, env):
        self._model = gp.Model(env=env, name='DSP')
        
        # create decision variables
        self._u1 = self._model.addVar(vtype=GRB.CONTINUOUS, name='u1')
        self._u2 = self._model.addVar(vtype=GRB.CONTINUOUS, name='u2')
        
        # create constraints
        self._model.addConstr(2*self._u1 + 4*self._u2 <= 8, name='c1')
        self._model.addConstr(3*self._u1 + 2*self._u2 <= 12, name='c2')
        self._model.addConstr(2*self._u1 + 3*self._u2 <= 10, name='c3')
        
        self._model.setObjective(1, GRB.MAXIMIZE)
        self._model.update()
        
        self._opt_obj = None
        self._opt_u1 = None
        self._opt_u2 = None
    
    def solve(self):
        print('-' * 50)
        print(f'Start solving dual subproblem.')
        self._model.optimize()
        
        status = None
        if self._model.status == GRB.OPTIMAL:
            self._opt_obj = self._model.objVal
            self._opt_u1 = self._u1.X
            self._opt_u2 = self._u2.X
            status = OptStatus.OPTIMAL
            print(f'\tdual subproblem is optimal.')
            print(f'\topt_obj={self._opt_obj:.2f}')
            print(f'\topt_y1={self._opt_u1:.2f}, opt_y2={self._opt_u2:.2f}')
        elif self._model.status == GRB.UNBOUNDED:
            status = OptStatus.UNBOUNDED
        else:
            status = OptStatus.ERROR
        
        print(f'Finish solving dual subproblem.')
        print('-' * 50)
        return status
    
    def update_objective(self, opt_y1, opt_y2):
        self._model.setObjective((300-4*opt_y1-5*opt_y2)*self._u1 + (220-2*opt_y1-3*opt_y2)*self._u2, GRB.MAXIMIZE)
        print(f'dual subproblem objective updated!')
    
    def clean_up(self):
        self._model.dispose()
        
    @property
    def opt_obj(self):
        return self._opt_obj
    
    @property
    def opt_u1(self):
        return self._opt_u1
    
    @property
    def opt_u2(self):
        return self._opt_u2

In [43]:
class BendersDecomposition:
    
    def __init__(self, master_solver, dual_subprob_solver):
        self._master_solver = master_solver
        self._dual_subprob_solver = dual_subprob_solver
        
    
    def optimize(self) -> OptStatus:
        eps = 1.0e-5
        lb = -np.inf
        ub = np.inf
        
        while True:
            # solve master problem
            master_status = self._master_solver.solve()
            if master_status == OptStatus.INFEASIBLE:
                return OptStatus.INFEASIBLE
            
            # update lower bound
            lb = np.max([lb, self._master_solver.opt_obj])
            print(f'Bounds: lb={lb:.2f}, ub={ub:.2f}')
            
            opt_y1 = self._master_solver.opt_y1
            opt_y2 = self._master_solver.opt_y2
            
            # solve subproblem
            self._dual_subprob_solver.update_objective(opt_y1, opt_y2)
            dsp_status = self._dual_subprob_solver.solve()
            
            if dsp_status == OptStatus.OPTIMAL:
                # update upper bound
                opt_obj = self._dual_subprob_solver.opt_obj
                ub = np.min([ub, 15*opt_y1 + 18*opt_y2 + opt_obj])
                print(f'Bounds: lb={lb:.2f}, ub={ub:.2f}')
                
                if ub - lb <= eps:
                    break
                
                opt_u1 = self._dual_subprob_solver.opt_u1
                opt_u2 = self._dual_subprob_solver.opt_u2
                self._master_solver.add_optimality_cut(opt_u1, opt_u2) 
            elif dsp_status == OptStatus.UNBOUNDED:
                opt_u1 = self._dual_subprob_solver.opt_u1
                opt_u2 = self._dual_subprob_solver.opt_u2
                self._master_solver.add_feasibility_cut(opt_u1, opt_u2) 
            

In [57]:
env = gp.Env('benders')
env.setParam("OutputFlag",0)
master_solver = MasterSolver(env)
dual_subprob_solver = DualSubprobSolver(env)

benders_decomposition = BendersDecomposition(master_solver, dual_subprob_solver)
benders_decomposition.optimize()

Set parameter Username
Set parameter LogFile to value "benders"
--------------------------------------------------
Start solving master problem.
	master problem is optimal.
	opt_obj=0.00
	opt_y1=0.00, opt_y2=0.00, opt_g=0.00
Finish solving master problem.
--------------------------------------------------
Bounds: lb=0.00, ub=inf
dual subproblem objective updated!
--------------------------------------------------
Start solving dual subproblem.
	dual subproblem is optimal.
	opt_obj=1200.00
	opt_y1=4.00, opt_y2=0.00
Finish solving dual subproblem.
--------------------------------------------------
Bounds: lb=0.00, ub=1200.00
Benders optimality cut added!
--------------------------------------------------
Start solving master problem.
	master problem is optimal.
	opt_obj=1080.00
	opt_y1=0.00, opt_y2=60.00, opt_g=0.00
Finish solving master problem.
--------------------------------------------------
Bounds: lb=1080.00, ub=1200.00
dual subproblem objective updated!
--------------------------

### A generic solver

In this section, we will create a more generic Benders decomposition based solver for linear programming problems.

In [59]:
class GenericLpMasterSolver:
    
    def __init__(self, f: np.array, B: np.array, b: np.array):
        # save data
        self._f = f
        self._B = B
        self._b = b 
        
        # env and model
        self._env = gp.Env('MasterEnv')
        # self._env.setParam("OutputFlag",0)
        self._model = gp.Model(env=self._env, name='MasterSolver')
        
        # create variables
        self._num_y_vars = len(f)
        self._y = self._model.addVars(self._num_y_vars, lb=0, vtype=GRB.CONTINUOUS, name='y')
        self._g = self._model.addVar(vtype=GRB.CONTINUOUS, lb=0, name='g')
        
        # create objective
        self._model.setObjective(gp.quicksum(self._f[i] * self._y.get(i) 
                                             for i in range(self._num_y_vars)) + self._g, 
                                 GRB.MINIMIZE)
        self._model.update()
        
        self._opt_obj = None
        self._opt_obj_y = None
        self._opt_y = None
        self._opt_g = None
        
    def solve(self) -> OptStatus:
        print('-' * 50)
        print(f'Start solving master problem.')
        print(self._model.display())
        self._model.optimize()
        
        opt_status = None
        if self._model.status == GRB.OPTIMAL:
            opt_status = OptStatus.OPTIMAL
            self._opt_obj = self._model.objVal
            self._opt_y = {
                i: self._y.get(i).X
                for i in range(self._num_y_vars)
            }
            self._opt_g = self._g.X
            self._opt_obj_y = self._opt_obj - self._opt_g
            print(f'\tmaster problem is optimal.')
            print(f'\topt_obj={self._opt_obj:.2f}')
            print(f'\topt_g={self._opt_g:.2f}')
            for i in range(self._num_y_vars):
                print(f'\topt_y{i}={self._opt_y.get(i)}')
        elif self._model.status == GRB.INFEASIBLE:
            print(f'\tmaster problem is infeasible.')
            opt_status = OptStatus.INFEASIBLE
        else:
            print(f'\tmaster problem encountered error.')
            opt_status = OptStatus.ERROR
        
        print(f'Finish solving master problem.') 
        print('-' * 50)
        return opt_status
    
    def add_feasibility_cut(self, opt_u: dict) -> None:
        constr_expr = [
            opt_u.get(u_idx) * (self._b[u_idx] - gp.quicksum(self._B[u_idx][j] * self._y.get(j) 
                                                   for j in range(self._num_y_vars)))
            for u_idx in opt_u.keys()
        ]
        self._model.addConstr(gp.quicksum(constr_expr) <= 0)
        print(f'Benders feasibility cut added!')
    
    def add_optimality_cut(self, opt_u: dict) -> None:
        constr_expr = [
            opt_u.get(u_idx) * (self._b[u_idx] - gp.quicksum(self._B[u_idx][j] * self._y.get(j) 
                                                   for j in range(self._num_y_vars)))
            for u_idx in opt_u.keys()
        ]
        self._model.addConstr(gp.quicksum(constr_expr) <= self._g)
        self._model.update()
        print(self._model.display())
        print(f'Benders optimality cut added!')
    
    def clean_up(self):
        self._model.dispose()
        self._env.dispose()
        
    @property
    def f(self):
        return self._f
        
    @property
    def opt_obj(self):
        return self._opt_obj
    
    @property
    def opt_obj_y(self):
        return self._opt_obj_y
    
    @property
    def opt_y(self):
        return self._opt_y
    
    @property
    def opt_g(self):
        return self._g

In [41]:
class GenericLpSubprobSolver:
    
    def __init__(self, A: np.array, c: np.array, B: np.array, b: np.array):
        # save data
        self._A = A 
        self._c = c 
        self._b = b 
        self._B = B
        
        # env and model
        self._env = gp.Env('SubprobEnv')
        self._env.setParam("OutputFlag",0)
        self._model = gp.Model(env=self._env, name='SubprobSolver')

        # create variables
        self._num_vars = len(b)
        self._u = self._model.addVars(self._num_vars, vtype=GRB.CONTINUOUS, name='u')

        # create constraints
        for c_idx in range(len(c)):
            self._model.addConstr(gp.quicksum(A[:,c_idx][i] * self._u.get(i) 
                                              for i in range(len(b))) <= c[c_idx])
        
        self._opt_obj = None
        self._opt_u = None
        
    def solve(self):
        print('-' * 50)
        print(f'Start solving dual subproblem.')
        self._model.optimize()
        
        status = None
        if self._model.status == GRB.OPTIMAL:
            self._opt_obj = self._model.objVal
            self._opt_u = {
                i: self._u.get(i).X
                for i in range(self._num_vars)
            }
            status = OptStatus.OPTIMAL
            print(f'\tdual subproblem is optimal.')
            print(f'\topt_obj={self._opt_obj:.2f}')
            for i in range(self._num_vars):
                print(f'\topt_u{i}={self._opt_u.get(i)}')
        elif self._model.status == GRB.UNBOUNDED:
            status = OptStatus.UNBOUNDED
        else:
            status = OptStatus.ERROR
        
        print(f'Finish solving dual subproblem.')
        print('-' * 50)
        return status

    def update_objective(self, opt_y: dict):
        obj_expr = [
            self._u.get(u_idx) * (self._b[u_idx] - sum(self._B[u_idx][j] * opt_y.get(j) 
                                                    for j in range(len(opt_y))))
            for u_idx in range(self._num_vars)
        ]
        self._model.setObjective(gp.quicksum(obj_expr), GRB.MAXIMIZE)
        print(f'dual subproblem objective updated!')
    
    def clean_up(self):
        self._model.dispose()
        self._env.dispose()
        
    @property
    def opt_obj(self):
        return self._opt_obj
    
    @property
    def opt_u(self):
        return self._opt_u

In [60]:
class GenericBendersSolver:
    
    def __init__(self, master_solver, dual_subprob_solver):
        self._master_solver = master_solver
        self._dual_subprob_solver = dual_subprob_solver
        
    
    def optimize(self,) -> OptStatus:
        eps = 1.0e-5
        lb = -np.inf
        ub = np.inf
        
        while True:
            # solve master problem
            master_status = self._master_solver.solve()
            if master_status == OptStatus.INFEASIBLE:
                return OptStatus.INFEASIBLE
            
            # update lower bound
            lb = np.max([lb, self._master_solver.opt_obj])
            print(f'Bounds: lb={lb:.2f}, ub={ub:.2f}')
            
            opt_y = self._master_solver.opt_y
            
            # solve subproblem
            self._dual_subprob_solver.update_objective(opt_y)
            dsp_status = self._dual_subprob_solver.solve()
            
            if dsp_status == OptStatus.OPTIMAL:
                # update upper bound
                opt_obj = self._dual_subprob_solver.opt_obj
                opt_obj_y = self._master_solver.opt_obj_y
                ub = np.min([ub, opt_obj_y + opt_obj])
                print(f'Bounds: lb={lb:.2f}, ub={ub:.2f}')
                
                if ub - lb <= eps:
                    break
                
                opt_u = self._dual_subprob_solver.opt_u
                self._master_solver.add_optimality_cut(opt_u) 
            elif dsp_status == OptStatus.UNBOUNDED:
                opt_u = self._dual_subprob_solver.opt_u
                self._master_solver.add_feasibility_cut(opt_u) 

In [61]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

c = np.array([8, 12, 10])
f = np.array([15, 18])
A = np.array([
    [2, 3, 2],
    [4, 2, 3]
])
B = np.array([
    [4, 5],
    [2, 3],
])
b = np.array([300, 220])

master_solver = GenericLpMasterSolver(f, B, b)
dual_subprob_solver = GenericLpSubprobSolver(A, c, B, b)

benders_solver = GenericBendersSolver(master_solver, dual_subprob_solver)
benders_solver.optimize()

Set parameter Username
Set parameter LogFile to value "MasterEnv"
Set parameter Username
Set parameter LogFile to value "SubprobEnv"
--------------------------------------------------
Start solving master problem.
Minimize
  15.0 y[0] + 18.0 y[1] + g
Subject To
None
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 0 rows, 3 columns and 0 nonzeros
Model fingerprint: 0xb00b2c2a
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 0 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  0.000000000e+00
	master 

### Implementation with callbacks